<a href="https://colab.research.google.com/github/guten-morgen3776/MUFG-Data-Science-Challenge-2025/blob/main/mufg_lgb_thr_avg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/signate mufg/train-3.csv')
test_data = pd.read_csv('/content/drive/MyDrive/signate mufg/test-2.csv')


In [ ]:
train_data.head(3)

,id,GrossApproval,SBAGuaranteedApproval,ApprovalFiscalYear,Subprogram,InitialInterestRate,FixedOrVariableInterestInd,TermInMonths,NaicsSector,CongressionalDistrict,BusinessType,BusinessAge,RevolverStatus,JobsSupported,CollateralInd,LoanStatus
0,1,23000,7093,2020,FA$TRK (Small Loan Express),11.92,V,84,Accommodation_food services,27,CORPORATION,Unanswered,0,1,N,1
1,2,104300,83477,2020,FA$TRK (Small Loan Express),8.40,V,102,Retail trade,14,CORPORATION,"Startup, Loan Funds will Open Business",1,1,N,1
2,3,163900,53246,2020,Contract Guaranty,4.32,F,119,Construction,0,CORPORATION,Existing or more than 2 years old,1,79,Y,0


In [ ]:
test_data.head(3)

,id,GrossApproval,SBAGuaranteedApproval,ApprovalFiscalYear,Subprogram,InitialInterestRate,FixedOrVariableInterestInd,TermInMonths,NaicsSector,CongressionalDistrict,BusinessType,BusinessAge,RevolverStatus,JobsSupported,CollateralInd
0,7553,22900,10607,2021,FA$TRK (Small Loan Express),4.30,F,5,Arts_entertainment_recreation,24,CORPORATION,Existing or more than 2 years old,0,10,Y
1,7554,29000,13390,2020,FA$TRK (Small Loan Express),6.97,V,120,Wholesale trade,5,INDIVIDUAL,New Business or 2 years or less,0,0,Y
2,7555,2913000,1737254,2020,Guaranty,5.61,V,200,Manufacturing,1,CORPORATION,"Startup, Loan Funds will Open Business",0,96,Y


In [ ]:
print(train_data.info())
print(test_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7552 entries, 0 to 7551
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          7552 non-null   int64  
 1   GrossApproval               7552 non-null   int64  
 2   SBAGuaranteedApproval       7552 non-null   int64  
 3   ApprovalFiscalYear          7552 non-null   int64  
 4   Subprogram                  7552 non-null   object 
 5   InitialInterestRate         7552 non-null   float64
 6   FixedOrVariableInterestInd  7552 non-null   object 
 7   TermInMonths                7552 non-null   int64  
 8   NaicsSector                 7552 non-null   object 
 9   CongressionalDistrict       7552 non-null   int64  
 10  BusinessType                7552 non-null   object 
 11  BusinessAge                 7552 non-null   object 
 12  RevolverStatus              7552 non-null   int64  
 13  JobsSupported               7552 

In [ ]:
train_data.describe()

,id,GrossApproval,SBAGuaranteedApproval,ApprovalFiscalYear,InitialInterestRate,TermInMonths,CongressionalDistrict,RevolverStatus,JobsSupported,LoanStatus
count,7552.000000,7.552000e+03,7.552000e+03,7552.000000,7552.000000,7552.000000,7552.000000,7552.000000,7552.000000,7552.000000
mean,3776.500000,7.219039e+05,4.536842e+05,2021.091499,7.379586,119.854211,12.681674,0.115731,14.281912,0.127648
std,2180.218949,1.112669e+06,7.805103e+05,1.125885,2.884902,82.423821,12.307653,0.319923,27.635214,0.333720
min,1.000000,5.000000e+03,2.500000e+03,2020.000000,1.030000,3.000000,0.000000,0.000000,0.000000,0.000000
25%,1888.750000,5.110000e+04,2.653525e+04,2020.000000,5.120000,60.000000,3.000000,0.000000,1.000000,0.000000
50%,3776.500000,1.896000e+05,1.063350e+05,2021.000000,6.610000,119.000000,7.000000,0.000000,5.000000,0.000000
75%,5664.250000,8.113000e+05,4.669272e+05,2022.000000,9.550000,120.000000,22.000000,0.000000,14.000000,0.000000
max,7552.000000,4.995000e+06,4.311817e+06,2024.000000,15.000000,306.000000,52.000000,1.000000,236.000000,1.000000


In [ ]:
train_data['GuaranteeRatio'] = train_data['SBAGuaranteedApproval'] / train_data['GrossApproval']
test_data['GuaranteeRatio'] = test_data['SBAGuaranteedApproval'] / test_data['GrossApproval']

In [ ]:
train_data['rate_term'] = train_data['InitialInterestRate'] * train_data['TermInMonths']
test_data['rate_term'] = test_data['InitialInterestRate'] * test_data['TermInMonths']

In [ ]:
train_data['TermInMonths'] = train_data['TermInMonths'].clip(train_data['TermInMonths'].quantile(0.01), train_data['TermInMonths'].quantile(0.99))
test_data['TermInMonths'] = test_data['TermInMonths'].clip(test_data['TermInMonths'].quantile(0.01), test_data['TermInMonths'].quantile(0.99))

In [ ]:
def bucket_term(x):
    if x <= 60:
        return "short"
    elif x <= 180:
        return "medium"
    else:
        return "long"

train_data["TermBucket"] = train_data["TermInMonths"].apply(bucket_term)
test_data["TermBucket"] = test_data["TermInMonths"].apply(bucket_term)


In [ ]:
amount_cols = ['GrossApproval','SBAGuaranteedApproval','JobsSupported']
for col in amount_cols:
    train_data[col] = np.log1p(train_data[col])
    test_data[col] = np.log1p(test_data[col])

In [ ]:
train_data.head(5)

,id,GrossApproval,SBAGuaranteedApproval,ApprovalFiscalYear,Subprogram,InitialInterestRate,FixedOrVariableInterestInd,TermInMonths,NaicsSector,CongressionalDistrict,BusinessType,BusinessAge,RevolverStatus,JobsSupported,CollateralInd,LoanStatus,GuaranteeRatio,rate_term,TermBucket
0,1,10.043293,8.867005,2020,FA$TRK (Small Loan Express),11.92,V,84,Accommodation_food services,27,CORPORATION,Unanswered,0,0.693147,N,1,0.308391,1001.28,medium
1,2,11.555036,11.332338,2020,FA$TRK (Small Loan Express),8.40,V,102,Retail trade,14,CORPORATION,"Startup, Loan Funds will Open Business",1,0.693147,N,1,0.800355,856.80,medium
2,3,12.007018,10.882697,2020,Contract Guaranty,4.32,F,119,Construction,0,CORPORATION,Existing or more than 2 years old,1,4.382027,Y,0,0.324869,514.08,medium
3,4,11.755095,11.258253,2021,Guaranty,6.40,F,59,Information,3,CORPORATION,Existing or more than 2 years old,0,1.098612,N,0,0.608446,377.60,short
4,5,14.571116,13.663458,2020,Guaranty,5.21,V,300,Health care_social assistance,44,CORPORATION,New Business or 2 years or less,0,1.791759,Y,0,0.403468,1563.00,long


In [ ]:
target = 'LoanStatus'
cat_cols = ["Subprogram", "FixedOrVariableInterestInd", "NaicsSector",
            "BusinessType", "BusinessAge", "CollateralInd","TermBucket"]
id_col = 'id'

In [ ]:
X = train_data.drop(columns=[target,'id'])
y = train_data[target]

In [ ]:
X = X.replace([np.inf, -np.inf], np.nan)

In [ ]:
for c in cat_cols:
  X[c] = X[c].astype('category')
  test_data[c] = test_data[c].astype('category')

In [ ]:
SEEDS = [42, 2024, 7, 99, 123, 512]
X_test = test_data.drop(columns=['id'])
X_test = X_test.replace([np.inf, -np.inf], np.nan)
oof_preds_seeds = []
test_preds_seeds = []
oof_true = y.copy()
all_fold_thrs = []

for sd in SEEDS:
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=sd)
    scores = []
    oof_preds = pd.Series(0,index=y.index, dtype=float)
    test_fold_preds = []
    fold_thrs = []

    for train_idx, valid_idx in skf.split(X, y):
      X_train,X_valid = X.iloc[train_idx],X.iloc[valid_idx]
      y_train,y_valid = y.iloc[train_idx],y.iloc[valid_idx]

      model = LGBMClassifier(
          objective ='binary',
          n_estimators=10000,
          class_weight='balanced',
          learning_rate=0.05,
          num_leaves=32,
          max_depth=-1,
          random_state=sd
      )
      model.fit(
          X_train,y_train,
          eval_set=[(X_valid,y_valid)],
          eval_metric='auc',
          callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)]
      )
      preds = model.predict_proba(X_valid,num_iteration=model.best_iteration_)[:,1]
      oof_preds.iloc[valid_idx] = preds
      #fold毎に閾値を最適化
      ths = np.linspace(0.05,0.95,181)
      f1s = [f1_score(y_valid, (preds > t).astype(int)) for t in ths]
      best_thr_fold = ths[int(np.argmax(f1s))]
      fold_thrs.append(best_thr_fold)

      test_fold_preds.append(
          model.predict_proba(X_test, num_iteration=model.best_iteration_)[:, 1]
      )

    mean_thr_seed = np.mean(fold_thrs)
    all_fold_thrs.extend(fold_thrs)
    print(f'[seed={sd}] fold mean thr={mean_thr_seed:.3f}')
    oof_preds_seeds.append(oof_preds.values)
    test_preds_seeds.append(np.mean(test_fold_preds, axis=0))



oof_seed_avg = np.mean(oof_preds_seeds, axis=0)
test_seed_avg = np.mean(test_preds_seeds, axis=0)

best_t = np.mean(all_fold_thrs)
best_f1 = f1_score(oof_true, (oof_seed_avg > best_t).astype(int))

print(f'[SEED平均]Best OOF F1: {best_f1:.5f} at thr={best_t:.3f}')



[LightGBM] [Info] Number of positive: 867, number of negative: 5929
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000923 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1707
[LightGBM] [Info] Number of data points in the train set: 6796, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Info] Number of positive: 867, number of negative: 5929
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001046 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1705
[LightGBM] [Info] Number of data points in the train set: 6796, number of used features: 17
[LightGBM] [Info] [binary:Bo

In [ ]:
test_pred_label = (test_seed_avg > best_t).astype(int)

In [ ]:
sample_sub = pd.read_csv('/content/drive/MyDrive/signate mufg/sample_submit.csv',header=None)

In [ ]:
sample_sub.head(3)

,0,1
0,7553,0
1,7554,0
2,7555,0


In [ ]:
sub_label = sample_sub.copy()
sub_label[1] = test_pred_label
sub_label.to_csv("mufg_submission_bucket_thr_avg.csv", index=False)


In [ ]:
sub_proba_test = sample_sub.copy()
sub_proba_test[1] = test_seed_avg
sub_proba_test.to_csv("test_lgb_category_bucket_thr_avg.csv", index=False)

In [ ]:
train_id = train_data['id'].copy()
sub_proba_train = pd.DataFrame({'id': train_id, 'oof': oof_seed_avg})
sub_proba_train.to_csv("train_lgb_category_bucket_thr_avg.csv", index=False)

In [ ]:
sub_label.head(3)

,0,1
0,7553,0
1,7554,0
2,7555,0


In [ ]:
sub_label.shape

(7552, 2)